# Summarization with Transformers

![](https://i.imgur.com/7SXKckD.png)

Transfer Learning is the power of leveraging already trained models and tune \ adapt them to our own downstream tasks.

# Automated document summarization methodologies

- __Extractive techniques:__ These methods use mathematical and statistical concepts to extract a key subset of content from the original document such that this subset contains the core information of the entire document. This content could be words, phrases or even sentences. The end result from this approach is a short executive summary of a couple of lines which are taken or extracted from the original document. No new content is generated in this technique hence the name "extractive".


- __Abstractive techniques:__ These methods are more complex and sophisticated and leverage language semantics to create representations and also make use of natural language generation (NLG) techniques where the machine makes use of knowledge bases and semantic representations to generate text on its own and create summaries just like a human would write them.

# Abstractive Summarization by Fine-tuning Transformers

In this section we’ll take a look at how Transformer models can be used to condense long documents into summaries, a task known as abstractive text summarization.

This is one of the most challenging NLP tasks as it requires a range of abilities, such as understanding long passages and generating coherent text that captures the main topics in a document.

However, when done well, text summarization is a powerful tool that can speed up various business processes by relieving the burden of domain experts to read long documents in detail.

Remember this is a sequence to sequence problem and requires past documents with the large and short (summary) form to make the model learn enough patterns to take in a new large document in the future and summarize it



## Install Relevant Libraries



In [ ]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You will be leveraging 🤗 Transformers and 🤗 Datasets as well as other dependencies

## Load Dataset

Here we load The CNN / DailyMail Dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. The current version supports both extractive and abstractive summarization, though the original version was created for machine reading and comprehension and abstractive question answering.

You can find the dataset in [HuggingFace Datasets](https://huggingface.co/datasets/cnn_dailymail)

For each instance, there is a string for the article, a string for the highlights, and a string for the id.

![](rhRwLI1.png)

In [ ]:
from datasets import load_dataset, load_metric

cnn_data = load_dataset("cnn_dailymail", '3.0.0')

100%|██████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 238.15it/s]


In [ ]:
cnn_data.keys()

dict_keys(['train', 'validation', 'test'])

In [ ]:
cnn_data['train'][:2]

{'article': ['LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office cha

Given infrastructure constraints we subset our dataset and limit training on only 10000 records.

In [ ]:
cnn_data['train'] = cnn_data['train'].shuffle(seed=42).select(range(10000))
cnn_data['validation'] = cnn_data['validation'].shuffle(seed=42).select(range(2000))
cnn_data['test'] = cnn_data['test'].shuffle(seed=42).select(range(2000))

Loading cached shuffled indices for dataset at /home/hp/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-380ef232430d96da.arrow
Loading cached shuffled indices for dataset at /home/hp/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-fe8ce4e4283768ec.arrow
Loading cached shuffled indices for dataset at /home/hp/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-914896ee6a47e4aa.arrow


The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
cnn_data

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 2000
    })
})

To access an actual element, you need to select a split first, then give an index:

In [ ]:
cnn_data["train"][0]

{'article': "By . Anthony Bond . PUBLISHED: . 07:03 EST, 2 March 2013 . | . UPDATED: . 08:07 EST, 2 March 2013 . Three members of the same family who died in a static caravan from carbon monoxide poisoning would have been unconscious 'within minutes', investigators said today. The bodies of married couple John and Audrey Cook were discovered alongside their daughter, Maureen, at the mobile home they shared on Tremarle Home Park in Camborne, west Cornwall. The inquests have now opened into the deaths last Saturday, with investigators saying the three died along with the family's pet dog, of carbon monoxide poisoning from a cooker. Tragic: The inquests have opened into the deaths of three members of the same family who were found in their static caravan last weekend. John and Audrey Cook are pictured . Awful: The family died following carbon monoxide poisoning at this caravan at the Tremarle Home Park in Camborne, Cornwall . It is also believed there was no working carbon monoxide detect

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(cnn_data["train"])

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the function `load_metric`.  

In [ ]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
metric = load_metric("rouge")
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

For summarization, one of the most commonly used metrics is the ROUGE score (short for Recall-Oriented Understudy for Gisting Evaluation).

The basic idea behind this metric is to compare a generated summary against a set of reference summaries that are typically created by humans.

To make this more precise, suppose we want to compare the following two summaries:

```
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"
```

One way to compare them could be to count the number of overlapping words, which in this case would be 6.

However, this is a bit crude, so instead ROUGE is based on computing the precision and recall scores for the overlap.

For ROUGE, recall measures how much of the reference summary is captured by the generated one. If we are just comparing words, recall can be calculated according to the following formula:

![](iYgPhYB.png)

For our simple example above, this formula gives a perfect recall of 6/6 = 1; i.e., all the words in the reference summary have been produced by the model.


This may sound great, but imagine if our generated summary had been “I really really loved reading the Hunger Games all night”. This would also have perfect recall, but is arguably a worse summary since it is verbose.

To deal with these scenarios we also compute the precision, which in the ROUGE context measures how much of the generated summary was relevant:

![](4aadAXM.png)


Applying this to our verbose summary gives a precision of 6/10 = 0.6, which is considerably worse than the precision of 6/7 = 0.86 obtained by our shorter one.

In practice, both precision and recall are usually computed, and then the F1-score (the harmonic mean of precision and recall) is reported.



In [1]:
#!wget http://i.imgur.com/rhRwLI1.png

In [ ]:
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"
scores = metric.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': AggregateScore(low=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923), mid=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923), high=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)),
 'rouge2': AggregateScore(low=Score(precision=0.6666666666666666, recall=0.8, fmeasure=0.7272727272727272), mid=Score(precision=0.6666666666666666, recall=0.8, fmeasure=0.7272727272727272), high=Score(precision=0.6666666666666666, recall=0.8, fmeasure=0.7272727272727272)),
 'rougeL': AggregateScore(low=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923), mid=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923), high=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)),
 'rougeLsum': AggregateScore(low=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923), mid=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.92307692307

🤗 Datasets actually computes confidence intervals for precision, recall, and F1-score; these are the low, mid, and high attributes you can see here.

Moreover, 🤗 Datasets computes a variety of ROUGE scores which are based on different types of text granularity when comparing the generated and reference summaries.

- rouge1 is the overlap of unigrams — this is just a fancy way of saying the overlap of words

- rouge2 measures the overlap between bigrams (think the overlap of pairs of words)

- rougeL and rougeLsum measure the longest matching sequences of words by looking for the longest common substrings in the generated and reference summaries

- The “sum” in rougeLsum refers to the fact that this metric is computed over a whole summary, while rougeL is computed as the average over individual sentences

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library.

Here we picked the [`t5-small`](https://huggingface.co/t5-small) checkpoint.

![](MFE2vfu.png)

T5 can be used for a variety of tasks and we will fine-tune it for summarization.

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [8774, 6, 48, 19, 3, 9, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this is a sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 19, 3, 9, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


/home/hp/.conda/envs/GenerataX/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


If you are using the T5 model, we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [ ]:
prefix = "summarize: "

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`.

This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
max_input_length = 1024
max_target_length = 128

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["highlights"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(cnn_data["train"][:2])

{'input_ids': [[21603, 10, 938, 3, 5, 11016, 12528, 3, 5, 3, 10744, 8775, 20619, 2326, 10, 3, 5, 10668, 10, 4928, 3, 6038, 6, 204, 1332, 2038, 3, 5, 1820, 3, 5, 3, 6880, 4296, 11430, 10, 3, 5, 12046, 10, 4560, 3, 6038, 6, 204, 1332, 2038, 3, 5, 5245, 724, 13, 8, 337, 384, 113, 3977, 16, 3, 9, 14491, 22133, 45, 4146, 1911, 6778, 15, 14566, 53, 133, 43, 118, 25429, 3, 31, 4065, 77, 676, 31, 6, 16273, 7, 243, 469, 5, 37, 5678, 13, 4464, 1158, 1079, 11, 31423, 6176, 130, 3883, 5815, 70, 3062, 6, 7758, 60, 35, 6, 44, 8, 1156, 234, 79, 2471, 30, 4691, 1635, 109, 1210, 1061, 16, 5184, 12940, 6, 4653, 26334, 5, 37, 16, 10952, 7, 43, 230, 2946, 139, 8, 14319, 336, 1856, 6, 28, 16273, 7, 2145, 8, 386, 3977, 590, 28, 8, 384, 31, 7, 3947, 1782, 6, 13, 4146, 1911, 6778, 15, 14566, 53, 45, 3, 9, 21859, 5, 21902, 447, 10, 37, 16, 10952, 7, 43, 2946, 139, 8, 14319, 13, 386, 724, 13, 8, 337, 384, 113, 130, 435, 16, 70, 14491, 22133, 336, 1851, 5, 1079, 11, 31423, 6176, 33, 3, 22665, 3, 5, 71, 210, 1329

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier.

This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = cnn_data.map(preprocess_function, batched=True)
no_deprecation_warning=True

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook.

The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data).

For instance, it will properly detect if you change the task in the first cell and rerun the notebook.

## Fine-tuning the Transformer Model

Now that our data is ready, we can download the pretrained model and fine-tune it.

Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class.

Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

To instantiate a `Seq2SeqTrainer`, we will need to define three more things.

The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training.

It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
#!pip install --upgrade accelerate
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

Here,

- we set the evaluation to be done at the end of each epoch
- tweak the learning rate
- use the `batch_size` defined at the top of the cell
- customize the weight decay

Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum.

Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions.

We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [ ]:
import time

In [ ]:
%%time
# train
trainer.train()

/home/hp/.conda/envs/GenerataX/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.872800,1.694398,24.369100,11.758400,20.091100,22.911900,19.000000
2,1.875000,1.692535,24.347200,11.719000,20.029200,22.901600,19.000000
3,1.865200,1.690754,24.429200,11.803600,20.106900,22.962900,19.000000


CPU times: user 26min 37s, sys: 12.7 s, total: 26min 50s
Wall time: 26min 49s


TrainOutput(global_step=1875, training_loss=1.8704330403645832, metrics={'train_runtime': 1609.1057, 'train_samples_per_second': 18.644, 'train_steps_per_second': 1.165, 'total_flos': 8120055539171328.0, 'train_loss': 1.8704330403645832, 'epoch': 3.0})

# Using your fine-tuned model for Summarization

Once you’ve fine-tuned the model you can use it with a pipeline object, for inference as follows:

In [ ]:
from transformers import pipeline

In [ ]:
model.to('cpu')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
summarize = pipeline(task='summarization', model=model, tokenizer=tokenizer)

In [ ]:
# An official announcement https://blog.google/technology/ai/gemini-collection/

document = """
Learn more about Gemini, our most capable AI model
Dec 06, 2023 3 articles
Gemini is a multimodal AI model, meaning it can process and generate different formats of data, including text, code, audio, images, and video.
This sets it apart from previous models, which were primarily focused on text-based tasks
Today we introduced Gemini, our largest and most capable AI model — and the next step on our journey toward making AI helpful for everyone. Built from the ground up to be multimodal, Gemini can generalize and seamlessly understand, operate across and combine different types of information, including text, images, audio, video and code. This means it has sophisticated multimodal reasoning and advanced coding capabilities. And with three different sizes — Ultra, Pro and Nano — Gemini has the flexibility to run on everything from data centers to mobile devices. We trained Gemini at scale on our AI-optimized infrastructure using Google's Tensor Processing Units (TPUs) v4 and v5e. Today, we also announced our most powerful and scalable TPU system to date, Cloud TPU v5p.
Gemini is available in some of our core products starting today: Bard is using a fine-tuned version of Gemini Pro for more advanced reasoning, planning, understanding and more. Pixel 8 Pro is the first smartphone engineered for Gemini Nano, using it in features like Summarize in Recorder and Smart Reply in Gboard. And we’re already starting to experiment with Gemini in Search, where it's making our Search Generative Experience (SGE) faster. Early next year, we’ll bring Gemini Ultra to a new Bard Advanced experience. And in the coming months, Gemini will power features in more of our products and services like Ads, Chrome and Duet AI.
Android developers who want to build Gemini-powered apps on-device can now sign up for an early preview of Gemini Nano, our most efficient model, via Android AICore. Starting December 13, developers and enterprise customers will be able to access Gemini Pro via the Gemini API in Vertex AI or Google AI Studio, our free web-based developer tool. And as we continue to refine Gemini Ultra, including completing extensive trust and safety checks, we’ll make it available to select groups before opening it up broadly to developers and enterprise customers early next year.
Explore the collection to learn more about our newest model, and the start of the Gemini era.
"""

In [ ]:
document

"\nLearn more about Gemini, our most capable AI model\nDec 06, 2023 3 articles\nGemini is a multimodal AI model, meaning it can process and generate different formats of data, including text, code, audio, images, and video. \nThis sets it apart from previous models, which were primarily focused on text-based tasks\nToday we introduced Gemini, our largest and most capable AI model — and the next step on our journey toward making AI helpful for everyone. Built from the ground up to be multimodal, Gemini can generalize and seamlessly understand, operate across and combine different types of information, including text, images, audio, video and code. This means it has sophisticated multimodal reasoning and advanced coding capabilities. And with three different sizes — Ultra, Pro and Nano — Gemini has the flexibility to run on everything from data centers to mobile devices. We trained Gemini at scale on our AI-optimized infrastructure using Google's Tensor Processing Units (TPUs) v4 and v5e

In [ ]:
authors_summary = """Gemini is a multimodal AI model, meaning it can process and generate different formats of data, including text, code, audio, images, and video.
This sets it apart from previous models, which were primarily focused on text-based tasks"""


In [ ]:
summary = summarize(document)[0]['summary_text']

In [ ]:
print('\n'.join(nltk.sent_tokenize(summary)))

Gemini is a multimodal AI model, meaning it can process and generate different formats of data, including text, code, audio, images, and video .
Built from the ground up to be multimodal, Gemini can generalize and seamlessly understand, operate across and combine different types of information .
This means it has sophisticated multimodal reasoning and advanced coding capabilities .


We can feed some examples from the test set (which the model has not seen) to our pipeline to get a feel for the quality of the summaries.

In [ ]:
for item in cnn_data['test'].shuffle(seed=42).select(range(10)):
  print('Acutal Headline:-\n', item['highlights'])
  print()
  summary = summarize(item['article'])[0]['summary_text']
  print('Summarized Headline:-\n', '\n'.join(nltk.sent_tokenize(summary)))
  print('\n')

Acutal Headline:-
 Battle between lenders has intensified recently, causing rates to plummet .
HSBC have now announced 1.99% interest deal on a five-year fix mortgage .
Offer expected to spark flood of rate cuts by banks and building societies .
Experts have described cheapest deal ever of its kind as 'astonishing'

Summarized Headline:-
 The battle between lenders has intensified in recent months, plunging home loan rates to their lowest in history .
But the mortgage wars will erupt again next week after HSBC announced a 1.99 per cent interest rate on a five-year fix .
Fifteen lenders had already cut rates across their ranges in the past week, but more are likely to follow .


Acutal Headline:-
 BBC's Back In Time For Dinner claims that grubs are the future of food .
The Robshaw family dig into cricket tacos, worm tarts and insect burgers .
Meat will become scarce or more expensive as demand for it grows .
Insects are full of protein, low in fat and packed full of nutrients .

Summari